## Importações e Configuração Inicial
 - AJUSTAR PARA BUSCAR INFORMAÇÕES DE PONTUAÇÕES MENSAIS

In [15]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
from cartolafc import CartolaFCError
import json
import time

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [16]:
# Função para buscar o nome do time a partir do ID
def buscar_nome_time_por_id(id_time):
    try:
        time_data = api.time(time_id=id_time)
        return time_data.info.nome  
    except CartolaFCError as e:
        print(f"Erro ao buscar nome para o ID {id_time}: {e}")
    except Exception as e:
        print(f"Erro inesperado para o ID {id_time}: {e}")
    return None

# ids_times = [13951133,18642587,49852616,3708025,44574236,528730,48498051,6148913,
#              3447341,117598,13707047,1273719,44810918,24468241,25401606,18661583,
#              29228373,17887202,20696550,212042,479510,4911779,14696986,47543456,
#              25565675,25811332,5823700,18421230,18346776,14227419,3015891,186283]

# ids_times = [29228373, 44574236, 20696550, 212042, 3708025, 18642587, 1273719, 186283,
#              18661583, 5823700, 50158297, 17887202, 25311459, 117598, 13707047, 4911779,
#              14696986, 13951133, 18223508, 18346776, 3447341, 49128587, 528730, 49960687,
#              25751748, 48498051, 44810918, 3851966, 14124559, 25565675, 335716, 25748736]

ids_times = [29228373,44574236,20696550,212042,3708025,18642587,1273719,186283,
             18661583,5823700,50158297,17887202,25311459,117598,13707047,4911779,
             14696986,18223508,18346776,3447341,49128587,528730,49960687,25751748,
             48498051,44810918,3851966,14124559,25565675,335716,25748736,1747619]

nomes_por_id = {}

# Loop para buscar os nomes
for id_ in ids_times:
    nome = buscar_nome_time_por_id(id_)
    if nome:
        nomes_por_id[id_] = nome
    time.sleep(0.5)  # respeita o rate limit

# Exibe os resultados
for id_, nome in nomes_por_id.items():
    print(f"{id_}: {nome}")


29228373: seralex
44574236: Analove10 ITAQUI GRANDE!!
20696550: Dom Camillo68
212042: Tatols Beants F.C
3708025: OlhaEleAiF.C!
18642587: Fedato Futebol Clube
1273719: Texas Club 2025
186283: FBC Colorado
18661583: pura bucha /botafogo
5823700: S.E.R. GRILLO
50158297: Lá do Itaqui
17887202: HS SPORTS F.C
25311459: FC Los Castilho 
117598: A Lenda Super Vasco F.c 
13707047: Super Vasco f.c
4911779: MAFRA MARTINS FC
14696986: VASCO MARTINS FC
18223508: Rolo Compressor  ZN
18346776: KP JUV.
3447341: PUXE FC
49128587: TORRESMO COM PINGA  
528730: Gremiomaniasm
49960687: Noah A 10
25751748: MauHumor F.C.
48498051: Pity10
44810918: lsauer fc
3851966: cartola scheuer
14124559: Paulo Virgili FC
25565675: FIGUEIRA DA ILHA
335716: teves_futsal20 f.c
25748736: C.A. Charru@
1747619: JV5 Tricolor Gaúcho


In [17]:
# nomes_por_id 
nomes_times = list(nomes_por_id.values())

# Para copiar e colar no seu código
print("nomes_times = [")
for nome in nomes_times:
    print(f'    "{nome}",')
print("]")

nomes_times = [
    "seralex",
    "Analove10 ITAQUI GRANDE!!",
    "Dom Camillo68",
    "Tatols Beants F.C",
    "OlhaEleAiF.C!",
    "Fedato Futebol Clube",
    "Texas Club 2025",
    "FBC Colorado",
    "pura bucha /botafogo",
    "S.E.R. GRILLO",
    "Lá do Itaqui",
    "HS SPORTS F.C",
    "FC Los Castilho ",
    "A Lenda Super Vasco F.c ",
    "Super Vasco f.c",
    "MAFRA MARTINS FC",
    "VASCO MARTINS FC",
    "Rolo Compressor  ZN",
    "KP JUV.",
    "PUXE FC",
    "TORRESMO COM PINGA  ",
    "Gremiomaniasm",
    "Noah A 10",
    "MauHumor F.C.",
    "Pity10",
    "lsauer fc",
    "cartola scheuer",
    "Paulo Virgili FC",
    "FIGUEIRA DA ILHA",
    "teves_futsal20 f.c",
    "C.A. Charru@",
    "JV5 Tricolor Gaúcho",
]


### Função para buscar o ID de um time pelo nome usando correspondência flexível

In [18]:
# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função para buscar o ID de um time pelo nome usando correspondência flexível
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]
        nome_proximo = get_close_matches(nome_time, nomes_api, n=1, cutoff=0.6)
        if nome_proximo:
            for time in times:
                if time.nome == nome_proximo[0]:
                    return time.id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar os IDs dos times pelo nome
for nome in nomes_times:
    time_id = buscar_id_time(nome)
    if time_id:
        ids_times[nome] = time_id
    else:
        print(f"Não foi possível encontrar o ID para o time {nome}")

# print(nomes_times)

In [19]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_eliminacao.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls)


✅ Arquivo salvo com sucesso: links_times_cartola_liga_eliminacao.xlsx


,Nome do Time,ID do Time,Link do Time
0,seralex,29228373,https://cartola.globo.com/#!/time/29228373
1,Analove10 ITAQUI GRANDE!!,44574236,https://cartola.globo.com/#!/time/44574236
2,Dom Camillo68,20696550,https://cartola.globo.com/#!/time/20696550
3,Tatols Beants F.C,212042,https://cartola.globo.com/#!/time/212042
4,OlhaEleAiF.C!,3708025,https://cartola.globo.com/#!/time/3708025
5,Fedato Futebol Clube,18642587,https://cartola.globo.com/#!/time/18642587
6,Texas Club 2025,1273719,https://cartola.globo.com/#!/time/1273719
7,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
8,pura bucha /botafogo,18661583,https://cartola.globo.com/#!/time/18661583
9,S.E.R. GRILLO,5823700,https://cartola.globo.com/#!/time/5823700


In [20]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual

print("Rodada Atual:", rodada_atual)

Rodada Atual: 20


In [21]:
# ===============================
#  LIGA ELIMINAÇÃO + PARCIAIS DA RODADA EM ANDAMENTO
# ===============================
import time, json, requests, pandas as pd

# -------- CONFIG --------
INICIO_ELIM = 20        # primeira rodada da Liga Eliminação
FIM_ELIM    = 38        # última rodada do Brasileirão
CAP_MULT    = 1.5       # multiplicador do capitão nas parciais
SLEEP_REQ   = 0.9       # pausa anti-rate-limit
MAX_RETRIES = 3
PREVIEW_ELIMINACAO_ATUAL = True  # só imprime possíveis eliminados da rodada atual (não remove)

# -------- CartolaFCError (fallback) --------
try:
    from cartolafc import CartolaFCError
except Exception:
    CartolaFCError = Exception

# -------- Sessão HTTP --------
sess = requests.Session()
sess.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
})

def http_status_e_rodada():
    r = sess.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    d = r.json()
    return int(d.get("status_mercado", 0)), int(d.get("rodada_atual", 0))

def get_parciais():
    r = sess.get("https://api.cartola.globo.com/atletas/pontuados", timeout=30)
    r.raise_for_status()
    return r.json().get("atletas", {})

def get_escalacao_por_rodada_http_dbg(time_id: int, rodada: int, max_retries=MAX_RETRIES):
    """
    Busca a escalação travada na rodada; se não vier, cai para 'última escalação'.
    Retorna: (atletas:list, capitao_id:int|None, http_status:int|None, motivo:str|None)
    """
    urls = [
        f"https://api.cartola.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartola.globo.com/time/id/{time_id}",
    ]
    last_err = None
    for url in urls:
        wait = 0.7
        for _ in range(max_retries):
            try:
                r = sess.get(url, timeout=30)
                st = r.status_code
                if st == 200:
                    d = r.json()
                    atletas = d.get("atletas") or []
                    return atletas, d.get("capitao_id"), st, (None if atletas else "sem_atletas")
                if st in (403, 429, 503):
                    time.sleep(wait); wait *= 1.8; continue
                last_err = f"status={st}"
                break
            except Exception as e:
                last_err = f"exc:{type(e).__name__}"
                time.sleep(wait); wait *= 1.8
    return [], None, None, last_err

# -------- Início --------
rodada_atual = api.mercado().rodada_atual
status_http, rodada_http = http_status_e_rodada()   # para saber se há parciais
rodada_ref = rodada_http or rodada_atual

# Verifica se a competição (R20) já começou
def campeonato_comecou(ids_times):
    for time_id in ids_times.values():
        try:
            p = api.time(time_id=time_id, rodada=INICIO_ELIM).ultima_pontuacao
            if p is not None:
                return True
        except CartolaFCError:
            continue
    return False

# -------- DataFrame base --------
if not campeonato_comecou(ids_times):
    print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {INICIO_ELIM}'])
    df_pontuacoes[:] = pd.NA
else:
    df_pontuacoes = pd.DataFrame(index=ids_times.keys())
    times_ativos = dict(ids_times)  # será reduzido pelos eliminados (rodadas ENCERRADAS)

    # -------- Rodadas ENCERRADAS (INICIO_ELIM .. rodada_atual-1) --------
    for rodada in range(INICIO_ELIM, min(rodada_atual, FIM_ELIM)):
        pontuacoes_rodada = {}
        print(f"🔄 Rodada {rodada} — Times participantes: {len(times_ativos)}")

        for nome, time_id in list(times_ativos.items()):
            try:
                t = api.time(time_id=time_id, rodada=rodada)
                pontuacoes_rodada[nome] = t.ultima_pontuacao
            except CartolaFCError as e:
                print(f"⚠️ Erro ao acessar pontuação da rodada {rodada} para {nome}: {e}")
                pontuacoes_rodada[nome] = None
            time.sleep(0.4)  # anti rate limit

        # adiciona a coluna da rodada encerrada
        df_pontuacoes[f'Rodada {rodada}'] = pd.Series(pontuacoes_rodada)

        # aplica eliminação APENAS em rodadas encerradas
        if len(pontuacoes_rodada) > 20 and rodada <= 37:
            rodada_series = pd.Series(pontuacoes_rodada).dropna()

            if rodada <= 28:
                eliminado = rodada_series.idxmin()
                print(f"❌ Eliminado da Rodada {rodada}: {eliminado}")
                times_ativos.pop(eliminado, None)

            elif rodada <= 37:
                eliminados = rodada_series.nsmallest(2).index.tolist()
                for eliminado in eliminados:
                    print(f"❌ Eliminado da Rodada {rodada}: {eliminado}")
                    times_ativos.pop(eliminado, None)

    # -------- Rodada EM ANDAMENTO (parciais) --------
    if status_http == 2 and INICIO_ELIM <= rodada_ref <= FIM_ELIM:
        col_atual = f"Rodada {rodada_ref}"
        print(f"🟡 Rodada {rodada_ref} em andamento — aplicando parciais aos {len(times_ativos)} times ainda ativos…")

        parciais_map = get_parciais()
        if not parciais_map:
            print("⚠️ Parciais vazias agora; tente novamente em instantes.")
        else:
            parciais_rodada = {}
            for nome, time_id in times_ativos.items():   # SOMENTE os ainda vivos
                try:
                    atletas, capitao_id, http_st, motivo = get_escalacao_por_rodada_http_dbg(int(time_id), rodada_ref)

                    # fallback via lib quando vier sem atletas
                    if not atletas and (motivo in ("sem_atletas", "sem_escalacao", None)):
                        try:
                            t = api.time(time_id=int(time_id), rodada=rodada_ref)
                            atletas = [{"atleta_id": a.atleta_id} for a in (t.atletas or [])]
                            if not capitao_id:
                                capitao_id = getattr(t, "capitao_id", None)
                        except Exception:
                            pass

                    total = 0.0
                    if atletas:
                        for a in atletas:
                            aid = str(a.get("atleta_id"))
                            if aid in parciais_map:
                                p = float(parciais_map[aid].get("pontuacao", 0) or 0)
                                if capitao_id and a.get("atleta_id") == capitao_id:
                                    p *= CAP_MULT
                                total += p
                    parciais_rodada[nome] = round(total, 2)
                    time.sleep(SLEEP_REQ)
                except Exception:
                    parciais_rodada[nome] = None

            # grava a coluna da rodada atual com as parciais (apenas vivos; eliminados ficam NaN)
            df_pontuacoes[col_atual] = pd.Series(parciais_rodada)

            # (opcional) Preview de eliminados desta rodada — NÃO remove
            if PREVIEW_ELIMINACAO_ATUAL and len(parciais_rodada) > 20 and rodada_ref <= 37:
                serie_ok = pd.Series(parciais_rodada).dropna()
                if not serie_ok.empty:
                    if rodada_ref <= 28:
                        prev = [serie_ok.idxmin()]
                    else:
                        prev = serie_ok.nsmallest(2).index.tolist()
                    print("🔎 Possíveis eliminados (preview, sem aplicar):", " | ".join(prev))
    else:
        print("ℹ️ Sem parciais para aplicar (status != 2 ou rodada fora do intervalo).")

# -------- Salvar / Exportar --------
excel_path = 'Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx'
df_pontuacoes.to_excel(excel_path)
print(f"✅ Arquivo Excel salvo: {excel_path}")

# Converte DF -> dict (NaN -> null no JS)
df_num  = df_pontuacoes.apply(pd.to_numeric, errors="coerce")
df_dict = df_num.where(pd.notna(df_num), None).to_dict(orient="index")

js_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao.js"
with open(js_path, "w", encoding="utf-8") as f:
    # Dados por rodada
    f.write("window.pontuacoesPorRodada = ")
    json.dump(df_dict, f, ensure_ascii=False, indent=2)
    f.write(";")

    # NEW: meta para o front saber se é PARCIAL (status_http=2) e qual rodada está rolando
    f.write("\nwindow.ligaEliminacaoMeta = ")
    json.dump({"status_http": int(status_http), "rodada": int(rodada_ref)}, f)
    f.write(";")

print(f"✅ Arquivo JS salvo: {js_path}")


# -------- Visualizar --------
try:
    display(df_pontuacoes.sort_values(by=list(df_pontuacoes.columns), ascending=False))
except Exception:
    print(df_pontuacoes.head())


🟡 Rodada 20 em andamento — aplicando parciais aos 32 times ainda ativos…
🔎 Possíveis eliminados (preview, sem aplicar): S.E.R. GRILLO
✅ Arquivo Excel salvo: Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx
✅ Arquivo JS salvo: Pontuacoes_Por_Rodada_Liga_Eliminacao.js


,Rodada 20
VASCO MARTINS FC,122.78
HS SPORTS F.C,60.86
A Lenda Super Vasco F.c,56.53
JV5 Tricolor Gaúcho,55.71
Tatols Beants F.C,55.71
Texas Club 2025,53.66
pura bucha /botafogo,52.81
TORRESMO COM PINGA,52.33
Super Vasco f.c,52.33
Analove10 ITAQUI GRANDE!!,47.81


In [22]:
# rodada_atual = api.mercado().rodada_atual

# # Verifica se o campeonato começou
# def campeonato_comecou(ids_times):
#     for time_id in ids_times.values():
#         try:
#             pontuacao = api.time(time_id=time_id, rodada=20).ultima_pontuacao
#             if pontuacao is not None:
#                 return True
#         except CartolaFCError:
#             continue
#     return False

# # Se campeonato não começou, cria tabela vazia
# if not campeonato_comecou(ids_times):
#     print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
#     total_rodadas = 19
#     df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(20, total_rodadas + 1)])
#     df_pontuacoes[:] = None
# else:
#     df_pontuacoes = pd.DataFrame()
#     times_ativos = dict(ids_times)  # cópia inicial

#     for rodada in range(20, rodada_atual):
#         pontuacoes_rodada = {}
#         print(f"🔄 Rodada {rodada} — Times participantes: {len(times_ativos)}")

#         for nome, time_id in times_ativos.items():
#             try:
#                 time_rodada = api.time(time_id=time_id, rodada=rodada)
#                 pontuacoes_rodada[nome] = time_rodada.ultima_pontuacao
#             except CartolaFCError as e:
#                 print(f"Erro ao acessar pontuação da rodada {rodada} para {nome}: {e}")
#                 pontuacoes_rodada[nome] = None
#             time.sleep(0.4)  # evita rate limit

#         # Adiciona pontuação da rodada
#         df_pontuacoes[f'Rodada {rodada}'] = pd.Series(pontuacoes_rodada)

#         # Eliminação
#         if len(pontuacoes_rodada) > 20 and rodada <= 37:
#             rodada_series = pd.Series(pontuacoes_rodada)
#             rodada_series = rodada_series.dropna()

#             if rodada <= 28:
#                 # Elimina 1 time
#                 eliminado = rodada_series.idxmin()
#                 print(f"❌ Eliminado da Rodada {rodada}: {eliminado}")
#                 times_ativos.pop(eliminado, None)

#             elif rodada <= 37:
#                 # Elimina 2 times
#                 eliminados = rodada_series.nsmallest(2).index.tolist()
#                 for eliminado in eliminados:
#                     print(f"❌ Eliminado da Rodada {rodada}: {eliminado}")
#                     times_ativos.pop(eliminado, None)

# # Salvar em Excel
# excel_path = 'Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx'
# df_pontuacoes.to_excel(excel_path)
# print(f"✅ Arquivo Excel salvo: {excel_path}")

# # Salvar em JS
# df_dict = df_pontuacoes.fillna("").to_dict(orient='index')
# js_content = f"const pontuacoesPorRodada = {json.dumps(df_dict, indent=2, ensure_ascii=False)},"

# js_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao.js"
# with open(js_path, "w", encoding="utf-8") as f:
#     f.write(js_content)

# print(f"✅ Arquivo JS salvo: {js_path}")

# # Exibir DataFrame final
# display(df_pontuacoes)


## Pontuação dos Finalistas/Campeões da Liga Eliminação

In [23]:
# Certifique-se de que o DataFrame df_pontuacoes já está carregado corretamente

# 1. Selecionar apenas as colunas até a Rodada 19 (caso existam)
rodadas_disponiveis = [col for col in df_pontuacoes.columns if "Rodada" in col]
rodadas_ate_38 = [col for col in rodadas_disponiveis if int(col.split()[-1]) <= 38]

# 2. Calcular a soma das pontuações até a Rodada 38
df_soma_ate_rodada_38 = df_pontuacoes[rodadas_ate_38].sum(axis=1)

# 3. Criar DataFrame com o total e classificar
df_classificacao_rodada_38 = pd.DataFrame({
    "Time": df_pontuacoes.index,
    "Total até Rodada 38": df_soma_ate_rodada_38
}).sort_values(by="Total até Rodada 38", ascending=False).reset_index(drop=True)

# 4. Exibir resultado
print(df_classificacao_rodada_38)


                         Time  Total até Rodada 38
0            VASCO MARTINS FC               122.78
1               HS SPORTS F.C                60.86
2    A Lenda Super Vasco F.c                 56.53
3         JV5 Tricolor Gaúcho                55.71
4           Tatols Beants F.C                55.71
5             Texas Club 2025                53.66
6        pura bucha /botafogo                52.81
7        TORRESMO COM PINGA                  52.33
8             Super Vasco f.c                52.33
9   Analove10 ITAQUI GRANDE!!                47.81
10            cartola scheuer                47.36
11       Fedato Futebol Clube                47.36
12           FC Los Castilho                 46.01
13                    seralex                45.26
14           MAFRA MARTINS FC                43.91
15         teves_futsal20 f.c                41.36
16        Rolo Compressor  ZN                40.21
17              OlhaEleAiF.C!                39.31
18                    KP JUV.  